# MVA - Homework 1 - Reinforcement Learning (2022/2023)

**Name:** PAILHAS Marceau

## Instructions

* The deadline is **November 10 at 11:59 pm (Paris time).**

* By doing this homework you agree to the late day policy, collaboration and misconduct rules reported on [Piazza](https://piazza.com/class/l4y5ubadwj64mb/post/6).

* **Mysterious or unsupported answers will not receive full credit**. A correct answer, unsupported by calculations, explanation, or algebraic work will receive no credit; an incorrect answer supported by substantially correct calculations and explanations might still receive partial credit.

* Answers should be provided in **English**.

# Colab setup

In [ ]:
from IPython import get_ipython

if 'google.colab' in str(get_ipython()):
  # install rlberry library
  !pip install git+https://github.com/rlberry-py/rlberry.git@mva2021#egg=rlberry[default] > /dev/null 2>&1

  # install ffmpeg-python for saving videos
  !pip install ffmpeg-python > /dev/null 2>&1

  # packages required to show video
  !pip install pyvirtualdisplay > /dev/null 2>&1
  !apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

  print("Libraries installed, please restart the runtime!")


In [ ]:
# Create directory for saving videos
!mkdir videos > /dev/null 2>&1

# Initialize display and import function to show videos
import rlberry.colab_utils.display_setup
from rlberry.colab_utils.display_setup import show_video

In [ ]:
# Useful libraries
import numpy as np
import matplotlib.pyplot as plt

# Preparation

In the coding exercises, you will use a *grid-world* MDP, which is represented in Python using the interface provided by the [Gym](https://gym.openai.com/) library. The cells below show how to interact with this MDP and how to visualize it.


In [ ]:
from rlberry.envs import GridWorld

def get_env():
  """Creates an instance of a grid-world MDP."""
  env = GridWorld(
      nrows=5,
      ncols=7,
      reward_at = {(0, 6):1.0},
      walls=((0, 4), (1, 4), (2, 4), (3, 4)),
      success_probability=0.9,
      terminal_states=((0, 6),)
  )
  return env

def render_policy(env, policy=None, horizon=50):
  """Visualize a policy in an environment

  Args:
    env: GridWorld
        environment where to run the policy
    policy: np.array
        matrix mapping states to action (Ns).
        If None, runs random policy.
    horizon: int
        maximum number of timesteps in the environment.
  """
  env.enable_rendering()
  state = env.reset()                       # get initial state
  for timestep in range(horizon):
      if policy is None:
        action = env.action_space.sample()  # take random actions
      else:
        action = policy[state]
      next_state, reward, is_terminal, info = env.step(action)
      state = next_state
      if is_terminal:
        break
  # save video and clear buffer
  env.save_video('./videos/gw.mp4', framerate=5)
  env.clear_render_buffer()
  env.disable_rendering()
  # show video
  show_video('./videos/gw.mp4')


In [ ]:
# Create an environment and visualize it
env = get_env()
render_policy(env)  # visualize random policy

# The reward function and transition probabilities can be accessed through
# the R and P attributes:
print(f"Shape of the reward array = (S, A) = {env.R.shape}")
print(f"Shape of the transition array = (S, A, S) = {env.P.shape}")
print(f"Reward at (s, a) = (1, 0): {env.R[1, 0]}")
print(f"Prob[s\'=2 | s=1, a=0]: {env.P[1, 0, 2]}")
print(f"Number of states and actions: {env.Ns}, {env.Na}")

# The states in the griworld correspond to (row, col) coordinates.
# The environment provides a mapping between (row, col) and the index of
# each state:
print(f"Index of state (1, 0): {env.coord2index[(1, 0)]}")
print(f"Coordinates of state 5: {env.index2coord[5]}")

# Part 1 - Dynamic Programming

## Question 1.1

Consider a general MDP with a discount factor of $\gamma < 1$. Assume that the horizon is infinite (so there is no termination). A policy $\pi$ in this MDP
induces a value function $V^\pi$. Suppose an affine transformation is applied to the reward, what is
the new value function? Is the optimal policy preserved?



### **Answer**

We apply an affine transformation to the reward, this means that $\forall (a,s)\in A \times S,\text{ }  r'(s,a) = c \times r(s,a) + d $.

<br>

Using the definition of the value function, $V^\pi$ is transformed in $V'^\pi$:
$$\begin{split}\forall s \in S,\text{ } V'^\pi (s) &= E\left[ \sum_{t=0}^\infty \gamma^t r'(s,a)|s_0 = s; a_t \sim d_t(h_t), \pi \right] & \text{ definition of $V^\pi$}
\\ &=E\left[ \sum_{t=0}^\infty \gamma^t  (c \times r(s,a) + d)|s_0 = s; a_t \sim d_t(h_t), \pi \right] & \text{ affine transformation of $r(s,a)$}
\\ &=c \times E\left[ \sum_{t=0}^\infty \gamma^t r(s_t,a_t)  \right] +d \times E\left[ \sum_{t=0}^\infty \gamma^t \right] & \text{ linearity of the expected value}\\
&= c \times V^\pi(s)+  \frac{d}{1- \gamma} & \text{ definition of $V^\pi$ and $E[cst]=cst$ }
\end{split} $$

<br>
<br>

For a given affine transformation of $r$ into $c\times r +d$, the value function $V^\pi$ is tranformed in $c \times V^\pi(s)+  \frac{d}{1- \gamma}$.

<br>

For a striclty positive $c$, we have $\max_{\pi}V'^\pi = c \times \max_{\pi}(V^\pi) +\frac{d}{1- \gamma}$; <br>
This means that an optimal policy that maximizes $V^\pi$ also maximizes $V'^\pi$. For a strictly positive c, the optimality of a policy is preserved.

<br>

If $c=0$, we have $\max_{\pi}V'^\pi = \frac{d}{1- \gamma}$; <br>
All policies maximize $V'^\pi$, which is not the case for $V^\pi$. So the optimality is not preserved.<br>
 For $c<0$, $ ~~\max_{\pi}V'^\pi = c \times \min_{\pi}(V^\pi) +\frac{d}{1- \gamma}$    ; so the optimal policy of $V^\pi $ is not the optimal policy of $V'^\pi$.




## Question 1.2

Consider an infinite-horizon $\gamma$-discounted MDP. We denote by $Q^*$ the $Q$-function of the optimal policy $\pi^*$. Prove that, for any function $Q(s, a)$ (which is **not** necessarily the value function of a policy), the following inequality holds for any state $s$:

$$
V^{\pi_Q}(s) \geq V^*(s) - \frac{2}{1-\gamma}||Q^*-Q||_\infty,
$$

where $||Q^*-Q||_\infty = \max_{s, a} |Q^*(s, a) - Q(s, a)|$ and $\pi_Q(s) \in \arg\max_a Q(s, a)$. Can you use this result to show that any policy $\pi$ such that $\pi(s) \in \arg\max_a Q^*(s, a)$ is optimal?

### **Answer**
1)$~~$We will first show the following result, $$\forall s \in S,~~~~ 2||Q^* - Q ||_\infty \geq Q^*(s, \pi^*(s))- Q^*(s, \pi_Q(s)) \text{ (*)}$$

<br>

To do so, we do a case disjunction,

<br>

Let $s \in S$,
 <br>

Suppose that $Q(s,\pi_Q(s)) \leq \frac{Q^*(s, \pi^*(s))+Q^*(s,\pi_Q(s))}{2}$

<br>

then $$\begin{align}Q^*(s,\pi^*(s))-Q(s,\pi_Q(s)) &\geq Q^*(s,\pi^*(s)) - (Q^*(s, \pi^*(s))+Q^*(s,\pi_Q(s)))/2 \\&= \frac{Q^*(s, \pi^*(s))-Q^*(s,\pi_Q(s))}{2}\end{align}$$

<br>

 $Q^*(s,\pi^*(s))-Q(s,\pi_Q(s)) = \max_{a \in A}Q^*(s,a)-\max_{a \in A}Q(s,a)\leq \max_{a \in A}(Q^*(s,a)-Q(s,a)) \leq \max_{(a,s) \in A\times S}(Q^*(s,a)-Q(s,a))= || Q^* - Q||_\infty $

<br>

We deduce that (*) for this case.

<br>

Suppose that $Q(s,\pi_Q(s)) \geq \frac{Q^*(s, \pi^*(s))+Q^*(s,\pi_Q(s))}{2}$ :

$Q(s,\pi_Q(s)) - Q^*(s,\pi_Q(s)) \geq \frac{Q^*(s, \pi^*(s))-Q^*(s,\pi_Q(s))}{2}$

<br>

We still have that $Q(s,\pi_Q(s)) - Q^*(s,\pi_Q(s)) = |Q(s,\pi_Q(s)) - Q^*(s,\pi_Q(s))|\leq || Q^* - Q||_\infty$

<br>

We deduce (*) for this second case.

<br>
<b/>We have proven (*).</b>

<br><br>
2)$~~$(*) can be rewritten $$\forall s \in S,~~~~ 2||Q^* - Q ||_\infty \geq Q^*(s, \pi^*(s))-Q^{\pi_Q}(s, \pi_Q(s))+Q^{\pi_Q}(s, \pi_Q(s))- Q^*(s, \pi_Q(s))$$
where $Q^{\pi_Q(s)}$ is the action-state function induced by $\pi_Q$ for the MDP $M$.

<br>

We have that $Q^*(s, \pi^*(s))= V^*(s)$ and $-Q^{\pi_Q}(s, \pi_Q(s))\geq -\max_a Q^{\pi_Q}(s, a)=- V^{\pi_Q}(s) $, thus $ Q^*(s, \pi^*(s))-Q^{\pi_Q}(s, \pi_Q(s))\geq V^*(s) - V^{\pi_Q}(s) $

<br>

Moreover, we have $Q^*(s, \pi_Q(s))- Q^{\pi_Q}(s, \pi_Q(s))= \gamma \sum_{y \in S}p(y|s,\pi_Q(s))(V^*(y)-V^{\pi_Q}(y)) $ (by definition of action-state value functions).

<br>
 We get that $Q^*(s, \pi_Q(s))- Q^{\pi_Q}(s, \pi_Q(s))\leq \gamma \sum_{y \in S}p(y|s,\pi_Q(s)) ||V^* - V^{\pi_Q}||_\infty  =  \gamma ||V^* - V^{\pi_Q}||_\infty$

<br>
<br>


 $$\forall s \in S, ~~Q^*(s, \pi_Q(s))- Q^{\pi_Q}(s, \pi_Q(s)) +2||Q^* - Q ||_\infty \geq Q^*(s, \pi^*(s))-Q^{\pi_Q}(s, \pi_Q(s))$$
which implies
$$\forall s \in S, ~~ \gamma ||V^* - V^{\pi_Q}||_\infty +2||Q^* - Q ||_\infty \geq V^*(s) - V^{\pi_Q}(s) ~~~~~~\text{(**)}$$
which implies $$\gamma ||V^* - V^{\pi_Q}||_\infty +2||Q^* - Q ||_\infty \geq ||V^* - V^{\pi_Q}||_\infty ~~~~~~\text{(***)}$$

<br>
3)$~~$We have to show that $$\forall s \in S, \forall n \in \mathbb{N}^*,~~ \gamma^n ||V^* - V^{\pi_Q}||_\infty +2||Q^* - Q ||_\infty \sum_{k=0}^{n-1}\gamma^k \geq V^*(s) - V^{\pi_Q}(s)$$
<br><br>

For the base case, $n=1$, it is true because it is (*).

<br>

For the inductive step, if it is true at $n$, as we have $$\gamma^n ||V^* - V^{\pi_Q}||_\infty \leq \gamma^n (\gamma ||V^* - V^{\pi_Q}||_\infty+2||Q^* - Q ||_\infty) $$ we get $$\gamma^{n+1} ||V^* - V^{\pi_Q}||_\infty +2||Q^* - Q ||_\infty \sum_{k=0}^{n}\gamma^k \geq V^*(s) - V^{\pi_Q}(s)$$

This completes the proof.

By making $n$ tend to $\infty$, we get $$V^*(s)- V^{\pi_Q}(s)\leq   \frac{2}{1-\gamma}||Q^*-Q||_\infty$$



<br>
<br>
If we have $\pi(s) \in \arg\max_a Q^*(s, a)$, then by changing $Q$ in $Q^*$, we have the following equation : $V^{\pi}(s) \geq V^*(s)$.
<br>
On the other hand, we have $V^{\pi}(s) \leq V^*(s)$ from the definition of the optimal policy.

<br>
Combining these two results, we get $V^{\pi}(s) = V^*(s)$. This means $\pi$ is an opimal policy.




## Question 1.3

In this question, you will implement and compare the policy and value iteration algorithms for a finite MDP.

Complete the functions `policy_evaluation`, `policy_iteration` and `value_iteration` below.


Compare value iteration and policy iteration. Highlight pros and cons of each method.

### **Answer**
The policy iteration converges in a few steps compared to value iteration. But for policy iteration, at each step, updating $\pi_k$ is more costly compared to value iteration.


In [ ]:
def policy_evaluation(P, R, policy, gamma, tol):
    """
    Args:
        P: np.array
            transition matrix (NsxNaxNs)
        R: np.array
            reward matrix (NsxNa)
        policy: np.array
            matrix mapping states to action (Ns)
        gamma: float
            discount factor
        tol: float
            precision of the solution
    Return:
        value_function: np.array
            The value function of the given policy
    """
    Ns, Na = R.shape
    # ====================================================
	  # YOUR IMPLEMENTATION HERE
    #
    value_function = np.zeros(Ns)
    value_function_iter = np.zeros(Ns)
    assertion = True


    while assertion:
      value_function = value_function_iter.copy()

      for s in range(Ns):
        value_function_iter[s]= R[s,policy[s]] + gamma*sum([P[s,policy[s],y]*value_function[y] for y in range(Ns)])


      assertion = max(abs(value_function_iter - value_function))>tol


    # ====================================================
    return value_function

In [ ]:
def policy_iteration(P, R, gamma, tol):
    """
    Args:
        P: np.array
            transition matrix (NsxNaxNs)
        R: np.array
            reward matrix (NsxNa)
        gamma: float
            discount factor
        tol: float
            precision of the solution
    Return:
        policy: np.array
            the final policy
        V: np.array
            the value function associated to the final policy
    """
    Ns, Na = R.shape
    V = np.zeros(Ns)
    policy = np.ones(Ns, dtype=int)
    # ====================================================
	  # YOUR IMPLEMENTATION HERE
    #
    policy_iter = np.zeros(Ns, dtype=int)
    V_maximize = np.zeros((Ns,Na))
    assertion = True

    while assertion  :

      V =  policy_evaluation(P, R, policy, gamma, tol)
      for a in range(Na):
        for s in range(Ns):
          V_maximize[s,a] = R[s,a] + gamma * sum(P[s,a,:]*V[:])



      policy_iter = policy.copy()
      policy = np.argmax(V_maximize, axis = 1)

      for s in range(Ns):
        assertion = not( (policy - policy_iter)[s] == 0)

        if assertion :
          break



    # ====================================================
    return policy, V

In [ ]:
def value_iteration(P, R, gamma, tol):
    """
    Args:
        P: np.array
            transition matrix (NsxNaxNs)
        R: np.array
            reward matrix (NsxNa)
        gamma: float
            discount factor
        tol: float
            precision of the solution
    Return:
        Q: final Q-function (at iteration n)
        greedy_policy: greedy policy wrt Qn
        Qfs: all Q-functions generated by the algorithm (for visualization)
    """
    Ns, Na = R.shape
    Q = np.zeros((Ns, Na))
    Qfs = [Q]
    assertion = True
    # ====================================================
	  # YOUR IMPLEMENTATION HERE
    #
    while assertion :


      Q_max = np.max(Q,axis= 1)

      Q = R + gamma*np.array([[sum(P[s,a,:]*Q_max[:])for a in range(Na) ] for s in range(Ns)])
      greedy_policy = np.argmax(Q, axis=1)

      Qfs.append(Q)
      assertion = np.max(abs(Qfs[-1]-Qfs[-2])) > tol


    # ====================================================
    return Q, greedy_policy, Qfs

### Testing your code

In [ ]:
# Parameters
tol = 1e-5
gamma = 0.99

# Environment
env = get_env()
Ns, Na = env.R.shape

# run value iteration to obtain Q-values
VI_Q, VI_greedypol, all_qfunctions = value_iteration(env.P, env.R, gamma, tol)

# render the policy
print("[VI]Greedy policy: ")
render_policy(env, VI_greedypol)

# compute the value function of the greedy policy using matrix inversion
# ====================================================
# YOUR IMPLEMENTATION HERE
#greedy_V = policy_evaluation(env.P, env.R, VI_greedypol, gamma, tol)
greedy_V = np.linalg.inv(np.eye(Ns)- gamma* np.array([[env.P[s,VI_greedypol[s],y] for y in range(Ns)]for s in range(Ns)])) @ np.array([env.R[s,VI_greedypol[s]] for s in range(Ns)])
# compute value function of the greedy policy
#

# ====================================================

# show the error between the computed V-functions and the final V-function
# (that should be the optimal one, if correctly implemented)
# as a function of time
final_V = all_qfunctions[-1].max(axis=1)
norms = [ np.linalg.norm(q.max(axis=1) - final_V) for q in all_qfunctions]
plt.plot(norms)
plt.xlabel('Iteration')
plt.ylabel('Error')
plt.title("Value iteration: convergence")

#### POLICY ITERATION ####
PI_policy, PI_V = policy_iteration(env.P, env.R, gamma, tol)
print("\n[PI]final policy: ")
render_policy(env, PI_policy)

## Uncomment below to check that everything is correct
assert np.allclose(PI_policy, VI_greedypol)
#     "You should check the code, the greedy policy computed by VI is not equal to the solution of PI"
assert np.allclose(PI_V, greedy_V, atol=0.001) #precision here had to be put to 0.001
#     "Since the policies are equal, even the value function should be"

plt.show()

# Part 2 - Tabular RL

## Question 2.1

The code below collects two datasets of transitions (containing states, actions, rewards and next states) for a discrete MDP.

For each of the datasets:

1. Estimate the transitions and rewards, $\hat{P}$ and $\hat{R}$.
2. Compute the optimal value function and the optimal policy with respect to the estimated MDP (defined by $\hat{P}$ and $\hat{R}$), which we denote by $\hat{\pi}$ and $\hat{V}$.
3. Numerically compare the performance of $\hat{\pi}$ and $\pi^\star$ (the true optimal policy), and the error between $\hat{V}$ and $V^*$ (the true optimal value function).

Which of the two data collection methods do you think is better? Why?

### **Answer**

The dataset 2 is the best, for dataset 1 it happens we have a very large error.

<br>

We learn something only when we reach the terminal state. When we sample randomly from all possible pairs $(s,a)$, we get the terminal state with probability $1/S=1/31$. For dataset 2, we must have at least 3 times this number, ie 100 samples work.

<br>

When we sample states for a given trajectory as for dataset 1, the terminal state is very less likely to be visited. When it is not visited, we don't update $R(s,a)$, then our computations lead to $Q(s,a)=0$.



In [ ]:
def get_random_policy_dataset(env, n_samples):
  """Get a dataset following a random policy to collect data."""
  states = []
  actions = []
  rewards = []
  next_states = []

  state = env.reset()
  for _ in range(n_samples):
    action = env.action_space.sample()
    next_state, reward, is_terminal, info = env.step(action)
    states.append(state)
    actions.append(action)
    rewards.append(reward)
    next_states.append(next_state)
    # update state
    state = next_state
    if is_terminal:
      state = env.reset()

  dataset = (states, actions, rewards, next_states)
  return dataset

def get_uniform_dataset(env, n_samples):
  """Get a dataset by uniformly sampling states and actions."""
  states = []
  actions = []
  rewards = []
  next_states = []
  for _ in range(n_samples):
    state = env.observation_space.sample()
    action = env.action_space.sample()
    next_state, reward, is_terminal, info = env.sample(state, action)
    states.append(state)
    actions.append(action)
    rewards.append(reward)
    next_states.append(next_state)

  dataset = (states, actions, rewards, next_states)
  return dataset


# Collect two different datasets
num_samples = 500 #2000
env = get_env()
dataset_1 = get_random_policy_dataset(env, num_samples)
dataset_2 = get_uniform_dataset(env, num_samples)


# Item 3: Estimate the MDP with the two datasets; compare the optimal value
# functions in the true and in the estimated MDPs
Ns, Na = env.R.shape
def Compute_Reward_and_Probability(dataset):
  "It returns the estimated probabilities P(Ns,Na,Ns) and estimated rewards R (Ns,Na)  "
  N_a_s = np.zeros((Ns,Na))
  R = np.zeros((Ns, Na))
  P = np.zeros((Ns,Na,Ns))

  states, actions, rewards, next_states = dataset
  for count in range(num_samples):
    s,a,y = states[count],actions[count], next_states[count]

    for x in range(Ns): #update values of P
      if x ==y:
        P[s,a,y] = (P[s,a,y]*N_a_s[s,a]+1)/(N_a_s[s,a]+1)

      else:
        P[s,a,x] = (P[s,a,x]*N_a_s[s,a])/(N_a_s[s,a]+1)

    R[s,a] = (R[s,a]*N_a_s[s,a]+rewards[count])/(N_a_s[s,a]+1) #update values of R

    N_a_s[s,a]+=1 #update the number of times action a was taken when in state s

  return P, R

P_1, R_1 = Compute_Reward_and_Probability(dataset_1) #this returns estimated probabilities and rewards for dataset 1
P_2, R_2 = Compute_Reward_and_Probability(dataset_2) #same for dataset 2

Q_1, pi_1 = value_iteration(P_1, R_1, gamma, tol)[:2] #we compute hat{V} and hat{pi} with environnement estimated with dataset1
Q_2, pi_2 = value_iteration(P_2, R_2, gamma, tol)[:2]

Q_star, pi_star = value_iteration(env.P, env.R, gamma, tol)[:2]

V_1 = np.max(Q_1, axis=1)
V_2 = np.max(Q_2, axis=1)
V_star = np.max(Q_star, axis=1)

print("The error between V* and hat{v} are: \n *", max(abs(V_1 - V_star)), " for the first dataset; \n *",max(abs(V_2 - V_star)), " for the second dataset; \n")

# ...

## Question 2.2

Suppose that $\hat{P}$ and $\hat{R}$ are estimated from a dataset of exactly $N$ i.i.d. samples from **each** state-action pair. This means that, for each $(s,a)$, we have $N$ samples $\{(s_1',r_1, \dots, s_N', r_N\}$, where $s_i' \sim P(\cdot | s,a)$ and $r_i \sim R(s,a)$ for $i=1,\dots,N$, and
$$ \hat{P}(s'|s,a) = \frac{1}{N}\sum_{i=1}^N \mathbb{1}(s_i' = s'), $$
$$ \hat{R}(s,a) = \frac{1}{N}\sum_{i=1}^N r_i.$$
Suppose that $R$ is a distribution with support in $[0,1]$. Let $\hat{V}$ be the optimal value function computed in the empirical MDP (i.e., the one with transitions $\hat{P}$ and rewards $\hat{R}$). For any $\delta\in(0,1)$, derive an upper bound to the error

$$ \| \hat{V} - V^* \|_\infty $$

which holds with probability at least $1-\delta$.

**Note** Your bound should only depend on deterministic quantities like $N$, $\gamma$, $\delta$, $S$, $A$. It should *not* dependent on the actual random samples.

**Hint** The following two inequalities may be helpful.

1. **A (simplified) lemma**. For any state $\bar{s}$,

$$ |\hat{V}(\bar{s}) - V^*(\bar{s})| \leq \frac{1}{1-\gamma}\max_{s,a} \left| R(s,a) - \hat{R}(s,a) + \gamma \sum_{s'}(P(s'|s,a) - \hat{P}(s'|s,a)) V^*(s') \right|$$

2. **Hoeffding's inequality**. Let $X_1, \dots X_N$ be $N$ i.i.d. random variables bounded in the interval $[0,b]$ for some $b>0$. Let $\bar{X} = \frac{1}{N}\sum_{i=1}^N X_i$ be the empirical mean. Then, for any $\epsilon > 0$,

$$ \mathbb{P}(|\bar{X} - \mathbb{E}[\bar{X}]| > \epsilon) \leq 2e^{-\frac{2N\epsilon^2}{b^2}}.$$

### **Answer**
1)
We have $||\hat{V}-V^*||_\infty \leq \frac{1}{1-\gamma}\max_{s,a} \left| R(s,a) - \hat{R}(s,a) + \gamma \sum_{s'}(P(s'|s,a) - \hat{P}(s'|s,a)) V^*(s') \right| :=\max_{s,a}M(s,a)$

<br>

We have $$\begin{align}\mathbb{P}(||\hat{V}-V^*||_\infty \leq \epsilon)&\geq \mathbb{P}(M \leq \epsilon)& \text{ inclusion of events}
\\&= \prod_{s,a}\mathbb{P}(M(s,a) \leq \epsilon) \end{align} $$

To ensure that the maximum $M$ is smaller than a value, you have to ensure that all components $M(s,a)$ are smaller than the same value.

<br>

We notice that $\mathbb{P}(M(s,a) \leq \epsilon)= 1 - \mathbb{P}(M(s,a) \geq \epsilon)$.  

<br><br>

2) We introduce $X_i=\frac{1}{1-\gamma}(r_i + \gamma \sum_{s'} 1(s_i' = s')V^*(s'))$ for $i \in \{1,\dots ,N\}$. We have the two following equalities.

<br>

$$\begin{align}(1-\gamma)\bar{X}=\frac{1}{N} \sum_i r_i + \gamma \sum_{s'}(\frac{1}{N}\sum_i  1(s_i' = s'))V^*(s') = \hat{R}(s,a)+\gamma \sum_{s'}\hat{P}(s'|s,a)V^*(s') &~~~~~~~~~~~~&\text{(1)}\\ &&\\(1-\gamma)\mathbb{E}[\bar{X}] = R(s,a) + \gamma \sum_{s'}P(s'|s,a)V^*(s')&& \text{(2)} \end{align}$$.

<br>

Combining (1) and (2),
we get $|\bar{X} - \mathbb{E}[\bar{X}]|= M(s,a)$.

<br>

3)We now have to show that $X_i$ is in $[0,\frac{1}{(1 - \gamma)^2}]$

<br>
We have, $V^*(s) = \mathbb{E}[\sum_{k=0}^\infty \gamma^k r(s,a)|s_0=s, \pi^*]\leq \mathbb{E}[ \sum_{k=0}^\infty \gamma^k] = \frac{1}{1-\gamma}$

We deduce $0 < V^*(s)< \frac{1}{1-\gamma}$ and by definition, $0 < r_i < 1 $.

$X_i>0$ is obvious. $X_i =\frac{1}{1-\gamma}( r_i + \gamma \sum_{s'}1(s_i=s')V(s')) \leq \frac{1}{1-\gamma}(1+  \gamma \frac{1}{1 - \gamma})= \frac{1}{(1-\gamma)^2}:=b$

<br>

4) $X_1,...,X_n$ are independent and identically distributed as the samples are iid.
We use Hoeffding's inequality:
$$ \mathbb{P}(M(s,a) > \epsilon) \leq 2e^{-\frac{2N\epsilon^2}{b^2}}.$$

<br>
We now can bound from below  $\prod_{s,a}\mathbb{P}(M(s,a) \leq \epsilon)$.

$\prod_{s,a}\mathbb{P}(M(s,a) \leq \epsilon)=\prod_{s,a}(1-\mathbb{P}(M(s,a) \geq \epsilon))\geq \prod_{s,a}(1- 2e^{-\frac{2N\epsilon^2}{b^2}})=1-\delta$.

<br>

<b>  


We get the following bound :$$\mathbb{P}\left(||\hat{V}-V^*||_\infty \leq \frac{1}{\sqrt{2N}(1-\gamma)^2}\sqrt{\ln{\left(\frac{2}{1-(1-\delta)^{1/AS}}\right)}} \right) \geq 1- \delta$$


## Question 2.3

Suppose once again that we are given a dataset of $N$ samples in the form of tuples $(s_i,a_i,s_i',r_i)$. We know that each tuple contains a valid transition from the true MDP, i.e., $s_i' \sim P(\cdot | s_i, a_i)$ and $r_i \sim R(s_i,a_i)$, while the state-action pairs $(s_i,a_i)$ from which the transition started can be arbitrary.

Suppose we want to apply Q-learning to this MDP. Can you think of a way to leverage this offline data to improve the sample-efficiency of the algorithm? What if we were using SARSA instead?

### **Answer**
The Sequential updates produce correlated samples. A good way to remedy that is to use experience replay.

Instead of sampling uniformely over the state - action to get $(s,a)$, you input a dataset. At each step, the sample $(a,s)$ is drawn from the dataset, the agent evolves to a new state $s'$.

This reinforces that the most recurring states for the MDP will be appearing many more times during the sampling. While the least recurring states for the MDP will be almost never sampled.


# Part 3 - RL with Function Approximation

## Question 3.1

Given a datset $(s_i, a_i, r_i, s_i')$ of (states, actions, rewards, next states), the Fitted Q-Iteration (FQI) algorithm proceeds as follows:


* We start from a $Q$ function $Q_0 \in \mathcal{F}$, where $\mathcal{F}$ is a function space;
* At every iteration $k$, we compute $Q_{k+1}$ as:

$$
Q_{k+1}\in\arg\min_{f\in\mathcal{F}} \frac{1}{2}\sum_{i=1}^N
\left(
  f(s_i, a_i) - y_i^k
\right)^2 + \lambda \Omega(f)
$$
where $y_i^k = r_i + \gamma \max_{a'}Q_k(s_i', a')$, $\Omega(f)$ is a regularization term and $\lambda > 0$ is the regularization coefficient.


Consider FQI with *linear* function approximation. That is, for a given feature map $\phi : S \rightarrow \mathbb{R}^d$, we consider a parametric family of $Q$ functions $Q_\theta(s,a) = \phi(s)^T\theta_a$ for $\theta_a\in\mathbb{R}^d$. Suppose we are applying FQI on a given dataset of $N$ tuples of the form $(s_i, a_i, r_i, s_i')$ and we are at the $k$-th iteration. Let $\theta_k \in\mathbb{R}^{d \times A}$ be our current parameter. Derive the *closed-form* update to find $\theta_{k+1}$, using $\frac{1}{2}\sum_a ||\theta_a||_2^2$ as regularization.

### **Answer**

$h:\theta =( \theta_1,\dots,\theta_A )\mapsto \frac{1}{2}\left[\sum_{i=1}^N
\left(
   \phi(s_i)^T\theta_{ a_i} - y_i^k
\right)^2 + \lambda \sum_a ||\theta_a||_2^2\right] $ is a convex and coercitive function. Then it admits a unique minimalizer on $\mathbb{R}^{dA}$.

<br>
We will compute the gradient of h with respect to $\theta$ and find the minimalizer $\theta^*$ for which $\nabla_\theta h(\theta^*)=\vec{0}$.

<br>

$$\nabla_{\theta_j} h = \sum_{i=1}^N
\left(
   \phi(s_i)^T\theta_{ a_i} - y_i^k
\right) \delta(a_i=j) \phi(s_i) + \lambda \theta_j$$

We derive $\theta^*$:
$$\begin{align} \nabla_{\theta_j} h =& 0\\ ⇔
(\lambda \mathbb{I}_d + \sum_{i=1}^N \delta(a_i = j)\phi(s_i)\phi(s_i)^T)\theta_j^*= &\sum_{i=1}^N y_i^k \delta(a_i=j)\phi(s_i)  \end{align}$$

<br>

$\phi(s_i)\phi(s_i)^T$ is a matrix with eigenvalues $||\phi(s_i)||_2^2$ and 0 so all its eigenvalues are positive.

<br>

A sum of positive-eingenvalue operators are also a positive-eigenvalue operator. This is because for positive-eigenvalue operators $A$, we have $\forall X \in \mathbb{R}^d$, $X^TAX \geq 0$. When we sum, we keep the bound from below by 0.

<br> Then $\lambda \mathbb{I}_d + \sum_{i=1}^N \delta(a_i = j)\phi(s_i)\phi(s_i)^T$ is invertible.

<br>
<br>



The actuation of $\theta$ is:
$$\theta_{k+1} = (\theta^*_1,\dots, \theta^*_A)$$  where $\theta_j$ is defined as:
$$\theta_j^*=
\left(\lambda \mathbb{I}_d + \sum_{i=1}^N \delta(a_i = j)\phi(s_i)\phi(s_i)^T \right)^{-1} \sum_{i=1}^N y_i^k \delta(a_i=j)\phi(s_i) $$


## Question 3.2

The code below creates a larger gridworld (with more states than the one used in the previous questions), and defines a feature map. Implement linear FQI to this environment (in the function `linear_fqi()` below), and compare the approximated $Q$ function to the optimal $Q$ function computed with value iteration.

Can you improve the feature map in order to reduce the approximation error?

### **Answer**

[explanation about how you tried to reduce the approximation error + FQI implementation below]

In [ ]:
def get_large_gridworld():
  """Creates an instance of a grid-world MDP with more states."""
  walls = [(ii, 10) for ii in range(15) if (ii != 7 and ii != 8)]
  env = GridWorld(
      nrows=15,
      ncols=15,
      reward_at = {(14, 14):1.0},
      walls=tuple(walls),
      success_probability=0.9,
      terminal_states=((14, 14),)
  )
  return env


class GridWorldFeatureMap:
  """Create features for state-action pairs

  Args:
    dim: int
      Feature dimension
    sigma: float
      RBF kernel bandwidth
  """
  def __init__(self, env, dim=15, sigma=0.25):
    self.index2coord = env.index2coord
    self.n_states = env.Ns
    self.n_actions = env.Na
    self.dim = dim
    self.sigma = sigma

    n_rows = env.nrows
    n_cols = env.ncols

    # build similarity matrix
    sim_matrix = np.zeros((self.n_states, self.n_states))
    for ii in range(self.n_states):
        row_ii, col_ii = self.index2coord[ii]
        x_ii = row_ii / n_rows
        y_ii = col_ii / n_cols
        for jj in range(self.n_states):
            row_jj, col_jj = self.index2coord[jj]
            x_jj = row_jj / n_rows
            y_jj = col_jj / n_cols
            dist = np.sqrt((x_jj - x_ii) ** 2.0 + (y_jj - y_ii) ** 2.0)
            sim_matrix[ii, jj] = np.exp(-(dist / sigma) ** 2.0)

    # factorize similarity matrix to obtain features
    uu, ss, vh = np.linalg.svd(sim_matrix, hermitian=True)
    self.feats = vh[:dim, :]

  def map(self, observation):
    feat = self.feats[:, observation].copy()
    return feat

In [ ]:
env = get_large_gridworld()
feat_map = GridWorldFeatureMap(env)

# Visualize large gridworld
render_policy(env)

# The features have dimension (feature_dim).
feature_example = feat_map.map(1) # feature representation of s=1
print(feature_example)

# Initial vector theta representing the Q function
theta = np.zeros((feat_map.dim, env.action_space.n))
print(theta.shape)
print(feature_example @ theta) # approximation of Q(s=1, a)

In [ ]:
def linear_fqi(env, feat_map, num_iterations, gamma,lambd):
  """
  # Linear FQI implementation
  # TO BE COMPLETED
  """
  N = 5000 #number of samples
  # get a dataset
  dataset = get_uniform_dataset(env, n_samples=N) #we chose the number of samples
  # OR dataset = get_random_policy_dataset(env, n_samples=...)

  (states, actions, rewards, next_states) = dataset

  theta = np.zeros((feat_map.dim, env.Na))
  Y = np.zeros(N)
  phi = feat_map.feats

  Z = np.zeros((feat_map.dim, env.Na))
  A = np.zeros((feat_map.dim, feat_map.dim, env.Na))
  #we compute only once the invert of the big matrix (lamba Id + sum phi(si)Tphi(si)), we call it A

  for i in range(N):
    A[:,:,actions[i]] = A[:,:,actions[i]] + np.outer(phi[:,states[i]],phi[:,states[i]])

  for action in range(Na):
    A[:, :, action] = A[:, :, action] + lambd * np.eye(feat_map.dim)
    A[:, :, action] = np.linalg.inv(A[:,:,action])


  for it in range(num_iterations):
    for i in range(N):
      Y[i] = rewards[i]+ gamma* np.max( phi[:,next_states[i]]@theta)

    for i in range(N):
      Z[:,actions[i]]+= Y[i]*phi[:,states[i]] #we computed sum yi^k phi(s_i)


    for action in range(Na):
      theta[:,action] = A[:,:,action] @ Z[:,action] #we update theta



  return theta

# ----------------------------
# Environment and feature map
# ----------------------------
gamma = 0.95
lambd = 0.1
env = get_large_gridworld()
# you can change the parameters of the feature map, and even try other maps!
feat_map = GridWorldFeatureMap(env, dim=150, sigma=0.5)

# -------
# Run FQI
# -------
theta = linear_fqi(env, feat_map,200, gamma,lambd )

# Compute and run greedy policy
Q_fqi = np.zeros((env.Ns, env.Na))
for ss in range(env.Ns):
  state_feat = feat_map.map(ss)
  Q_fqi[ss, :] = state_feat @ theta

V_fqi = Q_fqi.max(axis=1)
policy_fqi = Q_fqi.argmax(axis=1)
render_policy(env, policy_fqi, horizon=100)

# Visualize the approximate value function in the gridworld.
img = env.get_layout_img(V_fqi)
plt.imshow(img)
plt.show()

In [ ]:
phi = feat_map.feats
np.outer(phi[:,2],phi[:,2])[5,6]==phi[5,2]*phi[6,2]

In [ ]:
np.max(phi[:,2]@theta,axis=1)

In [ ]:
##It is what happens with value iteration

# run value iteration to obtain Q-values
VI_Q, VI_greedypol, all_qfunctions = value_iteration(env.P, env.R, gamma, tol)

# render the policy
print("[VI]Greedy policy: ")
render_policy(env, VI_greedypol)

VI_V = np.max(VI_Q, axis=1)

img = env.get_layout_img(np.array(VI_V))
plt.imshow(img)
plt.show()

In [ ]:
! jupyter nbconvert RL-MVA_2022-Homework_1_MarceauPAILHAS.ipnb --to pdf